# Experiment 01: Airline dataset

In this notebook we are going to perform a prediction experiment with XGBoost and LightGBM using the Airline dataset. We are also going to analyze if the dataset has [concept drift](https://en.wikipedia.org/wiki/Concept_drift). 

In this experiment we use [the airline dataset](http://kt.ijs.si/elena_ikonomovska/data.html) to predict arrival delay. The dataset consists of a large amount of records, containing flight arrival and departure details for all the commercial flights within the USA, from October 1987 to April 2008. Its size is around 116 million records and 5.76 GB of memory.

The machine we used was an [Azure NV24 VM](https://azure.microsoft.com/en-gb/blog/azure-n-series-general-availability-on-december-1/), which has [NVIDIA M60](http://www.nvidia.com/object/tesla-m60.html) GPUs. Its operating system is Ubuntu 16.04.

For both XGBoost and LightGBM we compiled from source, to get the last improvements. In XGboost we used the commit `6776292951565c8cd72e69afd9d94de1474f00c0` of May 26th. For LightGBM we used the commit `73968a96829e212b333c88cd44725c8c39c03ad1` of June 2nd. To get these versions and replicate our experiments:
```python
git clone --recursive *url_of_library*
git checkout *oldcommit*
git submodule update --recursive
```

In [1]:
import json
import sys
import warnings

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import pkg_resources
from IPython.display import SVG, display
from bokeh.io import export_svgs
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from libs.conversion import convert_cols_categorical_to_numeric, convert_related_cols_categorical_to_numeric
from libs.loaders import load_airline
from libs.notebook_memory_management import start_watching_memory
from libs.timer import Timer
from libs.utils import get_number_processors
from lightgbm.sklearn import LGBMRegressor, LGBMClassifier
from sklearn.metrics import (accuracy_score, roc_auc_score, f1_score, precision_score,
                             recall_score)
from libs.metrics import classification_metrics_binary, classification_metrics_binary_prob, binarize_prediction
from toolz import curry
from xgboost import XGBRegressor

print("System version: {}".format(sys.version))
print("XGBoost version: {}".format(pkg_resources.get_distribution('xgboost').version))
print("LightGBM version: {}".format(pkg_resources.get_distribution('lightgbm').version))

warnings.filterwarnings("ignore", category=DeprecationWarning)

Using TensorFlow backend.


System version: 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
XGBoost version: 0.6
LightGBM version: 0.2


The dataset we are going to use in this notebook is huge, therefore we want to monitor the memory consumption. 

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
start_watching_memory()

In [3] used 12.0859 MiB RAM in 15.70s, total RAM usage 270.79 MiB


# 1) XGBoost vs LightGBM benchmark
In the next section we compare both libraries speed, accuracy and other metrics for the dataset of airline arrival delay. 

### Data loading and management

In [4]:
%%time
df_plane = load_airline()
print(df_plane.shape)

INFO:libs.loaders:MOUNT_POINT not found in environment. Defaulting to /fileshare


(115069017, 14)
CPU times: user 1min 30s, sys: 14.2 s, total: 1min 44s
Wall time: 4min 12s
In [4] used 21987.2891 MiB RAM in 253.03s, total RAM usage 22258.07 MiB


In [5]:
df_plane.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,SFO,ORD,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,LAX,SFO,337,0,5
2,1987,10,1,4,5,35,HP,351,167,ICT,LAS,987,0,17
3,1987,10,1,4,5,40,DL,251,35,MCO,PBI,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,LAS,ORD,1515,0,17


In [5] used 0.2930 MiB RAM in 0.12s, total RAM usage 22258.37 MiB


The first step is to convert the categorical features to numeric features.

In [6]:
%%time
df_plane_numeric = convert_related_cols_categorical_to_numeric(df_plane, col_list=['Origin','Dest'])
del df_plane

CPU times: user 1min 34s, sys: 14.7 s, total: 1min 49s
Wall time: 1min 50s
In [6] used 5270.3789 MiB RAM in 110.32s, total RAM usage 27528.75 MiB


In [7]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,HP,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,DL,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,4,33,1515,0,17


In [7] used 0.0078 MiB RAM in 0.12s, total RAM usage 27528.75 MiB


In [8]:
%%time
df_plane_numeric = convert_cols_categorical_to_numeric(df_plane_numeric, col_list='UniqueCarrier')

CPU times: user 51.7 s, sys: 10.7 s, total: 1min 2s
Wall time: 1min 2s
In [8] used 13168.6133 MiB RAM in 63.03s, total RAM usage 40697.37 MiB


In [9]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17


In [9] used 0.0039 MiB RAM in 0.11s, total RAM usage 40697.37 MiB


To simplify the pipeline, we are going to set a classification problem where the goal is to classify wheather a flight has arrived delayed or not. For that we need to binarize the variable `ArrDelay`.

If you want to extend this experiment, you can set a regression problem and try to identify the number of minutes of delay a fight has. Both XGBoost and LightGBM have regression classes.

In [10]:
df_plane_numeric = df_plane_numeric.apply(lambda x: x.astype('int16'))

In [10] used 3072.6328 MiB RAM in 12.84s, total RAM usage 43770.00 MiB


In [11]:
%%time
df_plane_numeric['ArrDelayBinary'] = 1*(df_plane_numeric['ArrDelay'] > 0)

CPU times: user 264 ms, sys: 416 ms, total: 680 ms
Wall time: 676 ms
In [11] used 877.9727 MiB RAM in 0.78s, total RAM usage 44647.98 MiB


In [12]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay,ArrDelayBinary
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27,1
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5,1
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17,1
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2,0
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17,1


In [12] used 0.0000 MiB RAM in 0.11s, total RAM usage 44647.98 MiB


Once the features are prepared, let's split the dataset into train and test set. We won't use validation for this example (however, you can try to add it).

In [13]:
def split_train_val_test_df(df, val_size=0.2, test_size=0.2):
    train, validate, test = np.split(df.sample(frac=1), 
                                     [int((1-val_size-test_size)*len(df)), int((1-test_size)*len(df))])
    return train, validate, test

In [13] used 0.0000 MiB RAM in 0.10s, total RAM usage 44647.98 MiB


In [14]:
%%time
train, validate, test = split_train_val_test_df(df_plane_numeric, val_size=0, test_size=0.2)
print(train.shape)
print(validate.shape)
print(test.shape)

(92055213, 15)
(0, 15)
(23013804, 15)
CPU times: user 34.9 s, sys: 3.32 s, total: 38.3 s
Wall time: 38.6 s
In [14] used 4653.5234 MiB RAM in 38.73s, total RAM usage 49301.50 MiB


In [15]:
def generate_feables(df):
    X = df[df.columns.difference(['ArrDelay', 'ArrDelayBinary'])]
    y = df['ArrDelayBinary']
    return X,y

In [15] used 0.0000 MiB RAM in 0.10s, total RAM usage 49301.50 MiB


In [16]:
%%time
X_train, y_train = generate_feables(train)
X_val, y_val = generate_feables(validate)
X_test, y_test = generate_feables(test)

CPU times: user 880 ms, sys: 588 ms, total: 1.47 s
Wall time: 1.47 s
In [16] used 2853.2617 MiB RAM in 1.57s, total RAM usage 52154.76 MiB


In [17]:
del train, validate, test

In [17] used 0.0039 MiB RAM in 0.10s, total RAM usage 52154.77 MiB


### Training 
Now we are going to create two pipelines, one of XGBoost and one for LightGBM. The technology behind both libraries is different, so it is difficult to compare them in the exact same model setting. XGBoost grows the trees depth-wise and controls model complexity with `max_depth`. Instead, LightGBM uses a leaf-wise algorithm and controls the model complexity by `num_leaves`. As a tradeoff, we use XGBoost with `max_depth=8`, which will have max number leaves of 255, and compare it with LightGBM with `num_leaves=255`. 

In [18]:
results_dict = dict()

In [18] used 0.0000 MiB RAM in 0.10s, total RAM usage 52154.77 MiB


In [19]:
number_processors = get_number_processors()
print(number_processors)

24
In [19] used 0.0000 MiB RAM in 0.10s, total RAM usage 52154.77 MiB


Let's start with the XGBoost classifier.

In [ ]:
xgb_clf_pipeline = XGBRegressor(max_depth=8,
                                n_estimators=50,
                                min_child_weight=30,
                                learning_rate=0.1,
                                scale_pos_weight=2,
                                gamma=0.1,
                                reg_lambda=1,
                                subsample=1,
                                n_jobs=number_processors,
                                random_state=77)

In [20] used 0.0000 MiB RAM in 0.10s, total RAM usage 52154.77 MiB


In [ ]:
with Timer() as t:
    xgb_clf_pipeline.fit(X_train, y_train)

In [21] used 20410.4219 MiB RAM in 4478.31s, total RAM usage 72565.19 MiB


In [ ]:
results_dict['xgb']={
    'train_time': t.interval
}

In [22] used 0.0000 MiB RAM in 0.10s, total RAM usage 72565.19 MiB


Training XGBoost model with leave-wise growth

In [ ]:
xgb_hist_clf_pipeline = XGBRegressor(max_depth=0,
                                    n_estimators=50,
                                    min_child_weight=30,
                                    learning_rate=0.1,
                                    scale_pos_weight=2,
                                    gamma=0.1,
                                    reg_lambda=1,
                                    subsample=1,
                                    max_leaves=255,
                                    grow_policy='lossguide',
                                    tree_method='hist',
                                    n_jobs=number_processors,
                                    random_state=77)

In [23] used 0.0000 MiB RAM in 0.10s, total RAM usage 72565.19 MiB


In [ ]:
with Timer() as t:
    xgb_hist_clf_pipeline.fit(X_train, y_train)

In [24] used 23242.0391 MiB RAM in 389.66s, total RAM usage 95807.23 MiB


In [ ]:
results_dict['xgb_hist']={
    'train_time': t.interval
}

In [25] used 0.0039 MiB RAM in 0.10s, total RAM usage 95807.23 MiB


Training LightGBM model

In [ ]:
lgbm_clf_pipeline = LGBMRegressor(num_leaves=255,
                                  n_estimators=50,
                                  min_child_weight=30,
                                  learning_rate=0.1,
                                  scale_pos_weight=2,
                                  min_split_gain=0.1,
                                  reg_lambda=1,
                                  subsample=1,
                                  nthread=number_processors,
                                  seed=77)

In [26] used 0.0000 MiB RAM in 0.10s, total RAM usage 95807.23 MiB


In [ ]:
with Timer() as t:
    lgbm_clf_pipeline.fit(X_train, y_train)

In [27] used 2387.3750 MiB RAM in 261.93s, total RAM usage 98194.61 MiB


In [ ]:
results_dict['lgbm']={
    'train_time': t.interval
}

In [28] used 0.0156 MiB RAM in 0.10s, total RAM usage 98194.62 MiB


As it can be seen in the results, given the specific versions and parameters of both XGBoost and LightGBM and in this specific dataset, LightGBM is faster. 

In general terms, leaf-wise algorithms are more efficient, they converge much faster than depth-wise. However, it may cause over-fitting when the data is small or there are too many leaves.

### Evaluation
Now let's evaluate the model in the test set.

In [ ]:
with Timer() as t:
    y_prob_xgb = np.clip(xgb_clf_pipeline.predict(X_test), 0.0001, 0.9999)

In [29] used 178.8750 MiB RAM in 10.31s, total RAM usage 98373.50 MiB


In [ ]:
results_dict['xgb']['test_time'] = t.interval

In [30] used 0.0039 MiB RAM in 0.10s, total RAM usage 98373.50 MiB


In [ ]:
with Timer() as t:
    y_prob_xgb_hist = np.clip(xgb_hist_clf_pipeline.predict(X_test), 0.0001, 0.9999)

In [31] used 87.7930 MiB RAM in 11.36s, total RAM usage 98461.29 MiB


In [ ]:
results_dict['xgb_hist']['test_time'] = t.interval

In [32] used 0.0039 MiB RAM in 0.10s, total RAM usage 98461.30 MiB


In [ ]:
with Timer() as t:
    y_prob_lgbm = np.clip(lgbm_clf_pipeline.predict(X_test), 0.0001, 0.9999)

In [33] used 746.2188 MiB RAM in 11.34s, total RAM usage 99207.52 MiB


In [ ]:
results_dict['lgbm']['test_time'] = t.interval

In [34] used 0.0000 MiB RAM in 0.10s, total RAM usage 99207.52 MiB


### Metrics
We are going to obtain some metrics to evaluate the performance of each of the models.

In [ ]:
y_pred_xgb = binarize_prediction(y_prob_xgb)
y_pred_xgb_hist = binarize_prediction(y_prob_xgb_hist)
y_pred_lgbm = binarize_prediction(y_prob_lgbm)

In [35] used 529.0703 MiB RAM in 0.51s, total RAM usage 99736.59 MiB


In [ ]:
report_xgb = classification_metrics_binary(y_test, y_pred_xgb)
report2_xgb = classification_metrics_binary_prob(y_test, y_prob_xgb)
report_xgb.update(report2_xgb)

In [36] used 520.2227 MiB RAM in 27.38s, total RAM usage 100256.81 MiB


In [ ]:
results_dict['xgb']['performance'] = report_xgb

In [37] used 0.0078 MiB RAM in 0.10s, total RAM usage 100256.82 MiB


In [ ]:
report_xgb_hist = classification_metrics_binary(y_test, y_pred_xgb_hist)
report2_xgb_hist = classification_metrics_binary_prob(y_test, y_prob_xgb_hist)
report_xgb_hist.update(report2_xgb_hist)

In [38] used 13.8438 MiB RAM in 23.96s, total RAM usage 100270.66 MiB


In [ ]:
results_dict['xgb_hist']['performance'] = report_xgb_hist

In [39] used 0.0039 MiB RAM in 0.10s, total RAM usage 100270.66 MiB


In [ ]:
report_lgbm = classification_metrics_binary(y_test, y_pred_lgbm)
report2_lgbm = classification_metrics_binary_prob(y_test, y_prob_lgbm)
report_lgbm.update(report2_lgbm)

In [40] used 0.0000 MiB RAM in 24.98s, total RAM usage 100270.66 MiB


In [ ]:
results_dict['lgbm']['performance'] = report_lgbm

In [41] used 0.0039 MiB RAM in 0.10s, total RAM usage 100270.67 MiB


In [ ]:
# Results
print(json.dumps(results_dict, indent=4, sort_keys=True))

{
    "lgbm": {
        "performance": {
            "AUC": 0.8091311132268482,
            "Accuracy": 0.7361588288489812,
            "F1": 0.6951486933518146,
            "Precision": 0.76962356715726,
            "Recall": 0.6338156678407723
        },
        "test_time": 11.116355316989939,
        "train_time": 258.62174462400435
    },
    "xgb": {
        "performance": {
            "AUC": 0.7860779151938319,
            "Accuracy": 0.6347194492488074,
            "F1": 0.6992145582744992,
            "Precision": 0.5738919956705797,
            "Recall": 0.8945630456956335
        },
        "test_time": 10.082154828996863,
        "train_time": 4428.4945363200095
    },
    "xgb_hist": {
        "performance": {
            "AUC": 0.8077600605770576,
            "Accuracy": 0.6749219294645944,
            "F1": 0.7169589932187362,
            "Precision": 0.6109459971972688,
            "Recall": 0.8674878172778909
        },
        "test_time": 11.1422739700065,
        "

The experiment shows a fairly similar performance in both libraries, being LightGBM slightly better.

In [ ]:
del xgb_clf_pipeline, xgb_hist_clf_pipeline, lgbm_clf_pipeline, X_train, X_test, X_val

In [43] used -48640.3164 MiB RAM in 0.75s, total RAM usage 51630.35 MiB


# 2) Concept drift
In this section we are trying to find concept drift in the dataset to check if retraining is valuable.

### Data management
We are going to pack the data yearly to try to find concept drift

In [45]:
initial_year = 1987
num_ini = 5

In [45] used 0.0000 MiB RAM in 0.10s, total RAM usage 51630.35 MiB


In [46]:
def generate_subset_by_year(df, year_ini, year_end):
    return df[df['Year'].isin(range(year_ini, year_end))]

In [46] used 0.0000 MiB RAM in 0.10s, total RAM usage 51630.35 MiB


In [47]:
%%time
subset_base = generate_subset_by_year(df_plane_numeric, initial_year, initial_year + num_ini)
print(subset_base.shape)

(16810190, 15)
CPU times: user 31.2 s, sys: 3.02 s, total: 34.2 s
Wall time: 34.5 s
In [47] used 448.8945 MiB RAM in 34.63s, total RAM usage 52079.25 MiB


In [48]:
%%time
rest_df = df_plane_numeric.loc[df_plane_numeric.index.difference(subset_base.index)]
print(rest_df.shape)

(98258827, 15)
CPU times: user 51.5 s, sys: 4.64 s, total: 56.1 s
Wall time: 56.7 s
In [48] used 10267.1094 MiB RAM in 56.81s, total RAM usage 62346.36 MiB


### Traininig
Let's see what happens when we train on a subset of data and then evaluate in the data of the following years.

In [49]:
X_train, y_train = generate_feables(subset_base)
del(subset_base)

In [49] used -23161.6875 MiB RAM in 0.29s, total RAM usage 39184.67 MiB


In [50]:
clf = LGBMClassifier(num_leaves=255,
                    n_estimators=100,
                    min_child_weight=30,
                    learning_rate=0.1,
                    subsample=0.80,
                    colsample_bytree=0.80,
                    seed=42)

In [50] used 0.0000 MiB RAM in 0.10s, total RAM usage 39184.67 MiB


In [51]:
%%time
clf.fit(X_train, y_train)

CPU times: user 25min 51s, sys: 46.7 s, total: 26min 38s
Wall time: 1min 11s


LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.8, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=30, min_split_gain=0, n_estimators=100,
        nthread=-1, num_leaves=255, objective='binary', reg_alpha=0,
        reg_lambda=0, seed=42, silent=True, subsample=0.8,
        subsample_for_bin=50000, subsample_freq=1)

In [51] used 0.8555 MiB RAM in 71.49s, total RAM usage 39185.52 MiB


### Prediction

In [52]:
@curry
def predict_accuracy(clf, test_df):
    X_test, y_test = generate_feables(test_df)
    y_pred = clf.predict(X_test)
    return accuracy_score(y_test, y_pred)

In [52] used 0.0000 MiB RAM in 0.10s, total RAM usage 39185.52 MiB


In [53]:
%%time
accuracy_series = rest_df.groupby('Year').apply(predict_accuracy(clf))
print(accuracy_series)

Year
1992    0.757375
1993    0.756093
1994    0.744296
1995    0.731502
1996    0.722489
1997    0.720751
1998    0.706177
1999    0.700761
2000    0.689513
2001    0.674223
2002    0.680176
2003    0.686131
2004    0.680697
2005    0.673467
2006    0.663597
2007    0.651520
2008    0.636613
dtype: float64
CPU times: user 34min 20s, sys: 57.8 s, total: 35min 17s
Wall time: 1min 59s
In [53] used 2978.0977 MiB RAM in 119.97s, total RAM usage 42163.62 MiB


From the results we can observe that the accuracy of the model gets worse as the years pass on.

### Retraining
Now let's see what happens when we retrain and evaluate in the data of the following years.

In [54]:
new_init = 15

In [54] used 0.0000 MiB RAM in 0.10s, total RAM usage 42163.62 MiB


In [55]:
%%time
subset_retrain = generate_subset_by_year(df_plane_numeric, initial_year, initial_year + new_init) 
print(subset_retrain.shape)

(69425349, 15)
CPU times: user 1min 41s, sys: 6.93 s, total: 1min 48s
Wall time: 1min 49s
In [55] used 1854.1094 MiB RAM in 109.35s, total RAM usage 44017.73 MiB


In [56]:
X_train, y_train = generate_feables(subset_retrain)

In [56] used 1721.4414 MiB RAM in 0.96s, total RAM usage 45739.17 MiB


In [57]:
clf_retrain = LGBMClassifier(num_leaves=255,
                            n_estimators=100,
                            min_child_weight=30,
                            learning_rate=0.1,
                            subsample=0.80,
                            colsample_bytree=0.80,
                            seed=42)

In [57] used 0.0000 MiB RAM in 0.10s, total RAM usage 45739.17 MiB


In [58]:
%%time
clf_retrain.fit(X_train, y_train)

CPU times: user 1h 33min 5s, sys: 2min 4s, total: 1h 35min 10s
Wall time: 4min 19s


LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.8, learning_rate=0.1,
        max_bin=255, max_depth=-1, min_child_samples=10,
        min_child_weight=30, min_split_gain=0, n_estimators=100,
        nthread=-1, num_leaves=255, objective='binary', reg_alpha=0,
        reg_lambda=0, seed=42, silent=True, subsample=0.8,
        subsample_for_bin=50000, subsample_freq=1)

In [58] used 3576.0625 MiB RAM in 259.47s, total RAM usage 49315.23 MiB


### Prediction

In [59]:
%%time
rest_df = df_plane_numeric.loc[df_plane_numeric.index.difference(subset_retrain.index)]
print(rest_df.shape)

(45643668, 15)
CPU times: user 32 s, sys: 2.4 s, total: 34.4 s
Wall time: 33.4 s
In [59] used -1695.7891 MiB RAM in 33.54s, total RAM usage 47619.45 MiB


In [60]:
%%time
accuracy_retrain = rest_df.groupby('Year').apply(predict_accuracy(clf_retrain))
print(accuracy_retrain)

Year
2002    0.749985
2003    0.748284
2004    0.724928
2005    0.719791
2006    0.707446
2007    0.695424
2008    0.697949
dtype: float64
CPU times: user 13min 59s, sys: 50.3 s, total: 14min 49s
Wall time: 51.6 s
In [60] used -672.2148 MiB RAM in 51.70s, total RAM usage 46947.23 MiB


### Plot

In [61]:
def plot_metrics(metric1, metric2, legend1=None, legend2=None, x_label=None, y_label=None):
    lists = sorted(metric1.items()) 
    x, y = zip(*lists) 
    fig, ax = plt.subplots()
    ax.plot(x, y, label=legend1, color='#5975a4')
    lists2 = sorted(metric2.items()) 
    x2, y2 = zip(*lists2) 
    ax.plot(x2, y2, label=legend2, color='#5f9e6f')
    legend = ax.legend(loc=0)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    plt.show()
    return ax

In [61] used 0.0000 MiB RAM in 0.11s, total RAM usage 46947.23 MiB


In [62]:
from bokeh.models.sources import ColumnDataSource
from bokeh.models import HoverTool

In [62] used 0.0000 MiB RAM in 0.10s, total RAM usage 46947.23 MiB


In [63]:
data_cds = ColumnDataSource(pd.DataFrame({
    'train_acc': accuracy_series,
    'retrain_acc': accuracy_retrain
}))

In [63] used 0.1328 MiB RAM in 0.11s, total RAM usage 46947.36 MiB


In [64]:
# Airline Retrain Results
p = figure(y_axis_label='Accuracy', plot_width=700, plot_height=350, tools="pan,wheel_zoom,box_zoom,reset")
l1 = p.line('Year', 'train_acc', legend=' Train AUC', line_color="#5975a4", source=data_cds, line_width=6, line_cap="round")
p.line('Year', 'retrain_acc', legend=' Retrain AUC', line_color="#a1bae3", source=data_cds, line_width=6, line_cap="round")
l1_hover = HoverTool(renderers=[l1], tooltips=[( 'Train',  '@{train_acc}{0.4f}' ), ( 'Retrain',  '@{retrain_acc}{0.4f}' )], mode='vline')
p.add_tools(l1_hover)
show(p)

In [64] used 0.0000 MiB RAM in 0.18s, total RAM usage 46947.36 MiB


In [65]:
# SVG: Airline Retrain Results
# Save the plot and display it so that is is visible on Github
p.output_backend = "svg"
export_svgs(p, filename="airline_retrain.svg")
display(SVG('airline_retrain.svg'))

In [65] used -4539.0117 MiB RAM in 1.79s, total RAM usage 42408.35 MiB


As it can be seen, the performance is better after retraining. We have found concept drift in this dataset.